In [2]:
import requests
from bs4 import BeautifulSoup
import pandas as pd
import re
import time
import random

import collections

# Callable 에러 해결
# AttributeError: module 'collections' has no attribute 'Callable'
if not hasattr(collections, 'Callable'):
    collections.Callable = collections.abc.Callable

def clean_string(s):
    # 줄바꿈(\n, \r) 제거
    s = s.replace('\n', ' ').replace('\r', ' ')
    # 두 칸 이상의 공백을 하나의 공백으로 변경
    s = re.sub(r'\s{2,}', ' ', s)
    # 문자열 양쪽의 공백 제거
    return s.strip()

def crawl_list_in_info(data_dict,links):
    columns=['경력','학력','스킬','핵심역량','우대','기본우대','자격증','우대전공','외국어']
    for link in links:
        url = f'https://www.jobkorea.co.kr/{link}'

        # URL에서 HTML 컨텐츠 가져오기
        response = requests.get(url, headers={'User-Agent': 'Mozilla/5.0'})
        if response.status_code != 200:
            print(f"Failed to retrieve contents from {url}")
            return
        time.sleep(random.randint(3,5))
        # BeautifulSoup를 사용하여 HTML 파싱
        soup = BeautifulSoup(response.content, 'html.parser')

        print
        tbList = soup.find('div',{'class':'tbCol'}).find('dl',{'class':'tbList'})
        company_name = soup.find('div',{'class':'coInfo'}).find('h4').text
        print(company_name)
        dt_list = tbList.find_all('dt')
        dd_list = tbList.find_all('dd')
        data_list=['']*len(columns)
        for i in range(len(dd_list)):
            idx = columns.index(dt_list[i].text.strip())
            data_list[idx] = clean_string(dd_list[i].text)
        dd_list = list(map(lambda x:clean_string(x.text), dd_list))
        data_dict[company_name] = data_list

def crawl_list_link(data_dict, csv_filename, key=1):
    url = f'https://www.jobkorea.co.kr/Search/?duty=1000242&tabType=recruit&Page_No={key}'
    print(key)
    # URL에서 HTML 컨텐츠 가져오기
    response = requests.get(url, headers={'User-Agent': 'Mozilla/5.0'})
    if response.status_code != 200:
        print(f"Failed to retrieve contents from {url}")
        return
    
    # BeautifulSoup를 사용하여 HTML 파싱
    soup = BeautifulSoup(response.content, 'html.parser')
    
    # 필요한 데이터를 추출하여 리스트에 추가
    article = soup.find('article',{'class':'list'}) 
    if article:
        links = [header.attrs['href'] for header in article.find_all('a',{'class':'information-title-link'})]
        crawl_list_in_info(data_dict, links)
        crawl_list_link(data_dict, csv_filename, key+1)

def main():
    csv_filename = 'output.csv'
    # 데이터를 저장할 리스트 초기화
    data_dict = {}
    crawl_list_link(data_dict, csv_filename)
    df = pd.DataFrame.from_dict(data_dict,orient='index')
    df.columns = ['경력','학력','스킬','핵심역량','우대','기본우대','자격증','우대전공','외국어']
    df.index.name = '회사명'
    df.to_csv(csv_filename, encoding='utf-8-sig')
main()

1
현대자동차㈜
㈜아이엠뱅크
GS리테일
(주)우아한형제들
휴먼교육센터
SK매직
㈜하이미디어아카데미
라이나생명보험
스마트라이프텍㈜ 
㈜에이치비테크놀러지
기아㈜
티맵모빌리티㈜
재단법인 서울특별시 50플러스재단
(주)한백전자
㈜숲(SOOP CO., LTD.)
메가존클라우드㈜
카카오뱅크
㈜숲(SOOP CO., LTD.)
㈜숲(SOOP CO., LTD.)
㈜DYB최선어학원
2
스마트비전
㈜인포플라
㈜마음에이아이
㈜넥슨
에스디바이오센서㈜
㈜윕스
㈜커넥트웨이브
㈜멀티캠퍼스
㈜더블다운인터액티브
베스핀글로벌㈜
㈜넥슨
㈜천재교육
슈어소프트테크㈜
㈜비즈테크아이
㈜큐엠씨
㈜인콘
SK C&C
㈜아이유노글로벌
㈜에프앤가이드
카카오뱅크
3
로지스올그룹
현대건설
베스핀글로벌㈜
㈜케이디엑스한국데이터거래소
㈜비아이매트릭스
㈜비아이매트릭스
㈜비즈테크아이
㈜트윔
㈜앤씨앤
㈜헥토이노베이션
㈜아이유노글로벌
㈜웅진
코아텍㈜
㈜웅진
㈜이글루코퍼레이션
㈜넥슨
㈜마음에이아이
㈜마음에이아이
백금티앤에이
㈜제노레이
4
디아이티㈜
메가존클라우드㈜
현대오토에버㈜
㈜천재교과서
㈜플래티어
현대오토에버㈜
현대오토에버㈜
현대오토에버㈜
현대오토에버㈜
현대오토에버㈜
현대오토에버㈜
㈜미디어엑셀코리아
더치트주식회사
㈜한화
㈜한화
엔에이치엔㈜
주식회사 위메이드
금호전기㈜
㈜엑스와이지
㈜마음에이아이
5
㈜컴투스
㈜컴투스
엔에이치엔㈜
중고나라
㈜티맥스소프트
한국생산기술연구원
원드롭
㈜비바리퍼블리카
㈜이씨마이너
㈜이씨마이너
㈜이씨마이너
㈜넥슨
㈜넥슨
㈜넥슨
잡코리아(유)
㈜포비콘
제트서브
㈜딥엑스
㈜이음인터렉티브
스카이랩스
6
루미르㈜
㈜엑스코어시스템
㈜씨지인사이드
뉴로핏㈜
뉴로핏㈜
엑소시스템즈
퓨렌스㈜
㈜에프에스솔루션
㈜시노펙스
㈜딥노이드
㈜클라이원트
㈜아이엠에스나노텍
마크클라우드
㈜아이알에스소프트코리아
라이트비전㈜
㈜엘로이랩
㈜엘로이랩
씨이랩
㈜퍼브
시큐어포인트
7
휴먼교육센터
텔레픽스
메타빌드㈜
데이터랩㈜
㈜한국정보기술단
롯데이노베이트㈜
투아이시스㈜
로이드케이
녹십자홀딩스
씨와이오토텍(주)
㈜노르마
굿어스데이터㈜
굿어스데이터㈜


AttributeError: 'NoneType' object has no attribute 'find'